In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/soldv/python data/dengeu cases')

data1 = '/content/drive/MyDrive/soldv/python data/dengeu cases/curah hujan kotor.csv'

In [ ]:
df = pd.read_csv(data1, delimiter=',', header=None)
df.columns = ["tgl","curah hujan"]

df.head(15)

,tgl,curah hujan
0,01/01/2022,68.50
1,02/01/2022,3.30
2,03/01/2022,0.00
3,04/01/2022,"8,888.00"
4,05/01/2022,0.00
5,06/01/2022,3.30
6,07/01/2022,95.30
7,08/01/2022,0.00
8,09/01/2022,47.40
9,10/01/2022,13.30


In [ ]:
df.dtypes

,0
tgl,object
curah hujan,object


In [ ]:
df['curah hujan'] = df['curah hujan'].replace({'8,888.00': np.nan, '9,999.00': np.nan}).str.replace(',', '.')

In [ ]:
df['tgl'] = pd.to_datetime(df['tgl'], format='%d/%m/%Y')
# df['curah hujan'] = pd.to_numeric(df['curah hujan']) it's the same, just more attr
df['curah hujan'] = df['curah hujan'].astype(float)

df = df.set_index('tgl')
df.tail()

,curah hujan
tgl,
2023-12-27,39.7
2023-12-28,7.1
2023-12-29,35.2
2023-12-30,0.0
2023-12-31,0.0


In [ ]:
df.dtypes

,0
curah hujan,float64


In [ ]:
df['interp_rainf'] = df['curah hujan'].interpolate(method='time')
df['interp_rainf'][49]

<ipython-input-9-2057f39b6d6e>:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df['interp_rainf'][49]


28.5

In [ ]:
monthly_data = df.resample('M').agg({'interp_rainf': ['sum', 'count']})
monthly_data.columns = ['total_rainfall', 'days_count']

In [ ]:
monthly_data['mean_rainf'] = monthly_data['total_rainfall'] / monthly_data['days_count']

In [ ]:
monthly_data.head()

,total_rainfall,days_count,mean_rainf
tgl,,,
2022-01-31,426.40,31,13.754839
2022-02-28,370.15,28,13.219643
2022-03-31,125.55,31,4.050000
2022-04-30,145.10,30,4.836667
2022-05-31,199.10,31,6.422581


In [ ]:
monthly_data = monthly_data.reset_index()

# Extract year and month from the date
monthly_data['year'] = monthly_data['tgl'].dt.year
monthly_data['month'] = monthly_data['tgl'].dt.month

In [ ]:

# Drop the date column as it is no longer needed
monthly_data = monthly_data.drop(columns=['tgl', 'total_rainfall', 'days_count'], axis=1)
monthly_data.tail()

,mean_rainf,year,month
19,2.906452,2023,8
20,3.076667,2023,9
21,7.419355,2023,10
22,4.260000,2023,11
23,10.364516,2023,12


In [ ]:
import pandas as pd

# Read the Excel file
file_path = '/content/drive/MyDrive/soldv/python data/dengeu cases/jumlah-kasus-dbd-provinsi-jateng-2022-2023nov.xlsx'

# Initialize an empty list to collect data
all_data = []

month_mapping = {
    'Januari': 1, 'Februari': 2, 'Maret': 3, 'April': 4,
    'Mei': 5, 'Juni': 6, 'Juli': 7, 'Agustus': 8,
    'September': 9, 'Oktober': 10
}

# Process each sheet
for year in [2022, 2023]:
    # Read the specific sheet
    df2 = pd.read_excel(file_path, sheet_name=str(year))

    # Melt the data
    df_melted = pd.melt(df2, id_vars=['Wilayah', 'Penduduk'],
                        value_vars=['Januari', 'Februari', 'Maret', 'April', 'Mei', 'Juni', 'Juli', 'Agustus', 'September', 'Oktober'],
                        var_name='Month', value_name='Value')

    # Add the year column
    df_melted['Year'] = year

    df_melted['Month'] = df_melted['Month'].map(month_mapping)

    # Append the melted dataframe to the list
    all_data.append(df_melted)

# Concatenate all the data
final_df = pd.concat(all_data, ignore_index=True)

# Display the final DataFrame
print(final_df)

           Wilayah  Penduduk  Month  Value  Year
0         GROBOGAN   1470150      1    197  2022
1            DEMAK   1223217      1     56  2022
2         SEMARANG   1068492      1     17  2022
3       TEMANGGUNG    799764      1      7  2022
4           KENDAL   1033367      1     23  2022
..             ...       ...    ...    ...   ...
115          DEMAK   1185701     10     24  2023
116       SEMARANG   1084401     10      0  2023
117     TEMANGGUNG    784338     10      1  2023
118         KENDAL    982997     10     25  2023
119  KOTA SEMARANG   1885508     10      3  2023

[120 rows x 5 columns]


In [ ]:
monthly_data.head()

,mean_rainf,year,month
0,13.754839,2022,1
1,13.219643,2022,2
2,4.050000,2022,3
3,4.836667,2022,4
4,6.422581,2022,5


In [ ]:
# output_path = '/content/drive/MyDrive/soldv/python data/dengeu cases/curah hujan bulanan.csv'

# monthly_data.to_csv(output_path, index=False)

In [ ]:
output_path = '/content/drive/MyDrive/soldv/python data/dengeu cases/kasus dengeu.csv'

final_df.to_csv(output_path, index=False)